In [25]:
'''Trains a simple convnet on the MNIST dataset.
If implemented correctly, it is easy to get over 99.2% for testing accuracy
after a few epochs. And thre is still a lot of margin for parameter tuning,
e.g. different initializations, decreasing learning rates when the accuracy
stops increasing, or using model ensembling techniques, etc.
'''

from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential

from keras.layers import Activation, Dropout, Flatten, Dense

from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
#from sklearn import preprocessing
import numpy as np
#from sklearn.preprocessing import LabelEncoder
from keras.preprocessing.image import ImageDataGenerator

In [26]:
batch_size = 128
num_classes = 10
epochs = 20

# input image dimensions
img_rows, img_cols = 28, 28

# the data, shuffled and split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)



In [27]:
# In order to speed up the convergence, we may normalize the input values
# so that they are in the range of (0, 1) for (-1, 1)
# Your code here.
x_train = x_train.reshape(x_train.shape[0], img_rows * img_cols)
#x_train.shape
X_train = (x_train - np.mean(x_train, axis=1).reshape(-1,1)) / np.std(x_train, axis = 1).reshape(-1,1)
X_train = X_train.reshape(X_train.shape[0],img_rows,img_cols,1)

x_test = x_test.reshape(x_test.shape[0], img_rows * img_cols)
#x_train.shape
X_test = (x_test - np.mean(x_test, axis=1).reshape(-1,1)) / np.std(x_test, axis = 1).reshape(-1,1)
X_test = X_test.reshape(X_test.shape[0],img_rows,img_cols,1)

print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')



X_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [28]:
# Convert class vectors to binary class matrices, e.g. "1" ==> [0,1,0,0,0,0,0,0,0,0]
m_train = np.zeros([x_train.shape[0], num_classes])
m_test = np.zeros([x_test.shape[0], num_classes])
for i in range(y_train.shape[0]):
    m_train[i, y_train[i]] =1
y_train = m_train.copy()

for i in range(y_test.shape[0]):
    m_test[i, y_test[i]] =1
y_test = m_test.copy()
print(y_train.shape[0], 'train samples')
print(y_test.shape[0], 'test samples')

60000 train samples
10000 test samples


In [29]:
model = Sequential()

# Please build up a simple ConvNets by stacking a few conovlutioanl layers (kenel size with 3x3
# is a good choice, don't foget using non-linear activations for convolutional layers),
# max-pooling layers, dropout layers and dense/fully-connected layers.
# Your code here.
model.add(Conv2D(56,(3,3),input_shape=(28,28,1)))
model.add(Activation('relu'))
model.add(Conv2D(112,(3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(2,2))
model.add(Conv2D(112,(3,3)))
model.add(Activation('relu'))
model.add(Conv2D(224,(3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(2,2))
model.add(Flatten())

model.add(Dense(num_classes, activation='softmax'))

# complete the loss and optimizer
# Hints: use the cross-entropy loss, optimizer could be SGD or Adam, RMSProp, etc.
# Feel free to try different hyper-parameters.
model.compile(loss="categorical_crossentropy",
              optimizer="SGD",
              metrics=['accuracy'])

In [30]:


# Extra Points 1: use data augmentation in the progress of model training.
# Note that data augmentation is a practical technique for reducing overfitting.
# Hints: you may refer to https://keras.io/preprocessing/image/
# Your code here
# x_train = data_augment(...)
datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True)

datagen.fit(X_train)
model.fit_generator(datagen.flow(X_train,y_train,batch_size=32),
                   steps_per_epoch= len(X_train)/ 32, epochs = epochs)

score = model.evaluate(X_test, y_test, verbose=0)
print (score)

Epoch 1/20
1875/1875 [==============================] - 35s - loss: 1.2422 - acc: 0.5701    
Epoch 2/20
1875/1875 [==============================] - 34s - loss: 0.5367 - acc: 0.8262    
Epoch 3/20
1875/1875 [==============================] - 35s - loss: 0.3848 - acc: 0.8772    
Epoch 4/20
1875/1875 [==============================] - 35s - loss: 0.3171 - acc: 0.9000    
Epoch 5/20
1875/1875 [==============================] - 34s - loss: 0.2641 - acc: 0.9167    
Epoch 6/20
1875/1875 [==============================] - 34s - loss: 0.2351 - acc: 0.9256    
Epoch 7/20
1875/1875 [==============================] - 34s - loss: 0.2082 - acc: 0.9338    
Epoch 8/20
1875/1875 [==============================] - 34s - loss: 0.1943 - acc: 0.9389    
Epoch 9/20
1875/1875 [==============================] - 34s - loss: 0.1756 - acc: 0.9449    
Epoch 10/20
1875/1875 [==============================] - 34s - loss: 0.1635 - acc: 0.9497    
Epoch 11/20
1875/1875 [==============================] - 34s - loss: 

In [36]:
# Extra Points 2: use K-Fold cross-validation for ensembling k models,
# i.e. (1) split the whole training data into K folds;
#      (2) train K models based on different training data;
#      (3) when evaludating the testing data, averaging over K model predictions as final output.
# The codes may look like:
#   for i in range(K):
#       x_train, y_train = ...
#       model_i = train(x_train , y_train)

#k = 10
score = []

n = np.array_split( range(x_train.shape[0]), 10)
for i in range(10):
    models = keras.models.clone_model(model)
    models.compile(loss="categorical_crossentropy",
              optimizer="SGD",
              metrics=['accuracy'])
    x_traint = X_train[n[i], :]
    y_traint = y_train[n[i], :]
    
    datagen = ImageDataGenerator(
        featurewise_center=True,
        featurewise_std_normalization=True,
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        horizontal_flip=True)
    
    datagen.fit(X_train)
    models.fit_generator(datagen.flow(X_train,y_train,batch_size=32),
                   steps_per_epoch= len(X_train)/ batch_size, epochs = epochs)
#    model.fit(x_traint, y_traint,
#          batch_size=batch_size,
#          epochs=epochs,
#          verbose=1,
#          validation_data=(x_test, y_test))
    score.append( models.evaluate(X_test, y_test, verbose=0))
    



Epoch 1/20
469/468 [==============================] - 9s - loss: 2.1365 - acc: 0.2253     
Epoch 2/20
469/468 [==============================] - 8s - loss: 1.4130 - acc: 0.5135     
Epoch 3/20
469/468 [==============================] - 8s - loss: 0.9876 - acc: 0.6671     
Epoch 4/20
469/468 [==============================] - 8s - loss: 0.7800 - acc: 0.7370     
Epoch 5/20
469/468 [==============================] - 8s - loss: 0.6546 - acc: 0.7868     
Epoch 6/20
469/468 [==============================] - 8s - loss: 0.5683 - acc: 0.8119     
Epoch 7/20
469/468 [==============================] - 8s - loss: 0.5162 - acc: 0.8305     
Epoch 8/20
469/468 [==============================] - 8s - loss: 0.4590 - acc: 0.8464     
Epoch 9/20
469/468 [==============================] - 8s - loss: 0.4050 - acc: 0.8666     
Epoch 10/20
469/468 [==============================] - 8s - loss: 0.3942 - acc: 0.8737     
Epoch 11/20
469/468 [==============================] - 8s - loss: 0.3627 - acc: 0.8833   

469/468 [==============================] - 8s - loss: 0.4041 - acc: 0.8667     
Epoch 11/20
469/468 [==============================] - 8s - loss: 0.3849 - acc: 0.8762     
Epoch 12/20
469/468 [==============================] - 8s - loss: 0.3507 - acc: 0.8875     
Epoch 13/20
469/468 [==============================] - 8s - loss: 0.3534 - acc: 0.8886     
Epoch 14/20
469/468 [==============================] - 8s - loss: 0.3286 - acc: 0.8935     
Epoch 15/20
469/468 [==============================] - 8s - loss: 0.3170 - acc: 0.8983     
Epoch 16/20
469/468 [==============================] - 8s - loss: 0.3016 - acc: 0.9043     
Epoch 17/20
469/468 [==============================] - 8s - loss: 0.2937 - acc: 0.9071     
Epoch 18/20
469/468 [==============================] - 8s - loss: 0.2769 - acc: 0.9120     
Epoch 19/20
469/468 [==============================] - 9s - loss: 0.2694 - acc: 0.9126     
Epoch 20/20
469/468 [==============================] - 8s - loss: 0.2730 - acc: 0.9138     


469/468 [==============================] - 8s - loss: 0.2670 - acc: 0.9144     
Epoch 20/20
469/468 [==============================] - 9s - loss: 0.2586 - acc: 0.9170     
Epoch 1/20
469/468 [==============================] - 9s - loss: 2.1808 - acc: 0.2054     
Epoch 2/20
469/468 [==============================] - 8s - loss: 1.4909 - acc: 0.4814     
Epoch 3/20
469/468 [==============================] - 8s - loss: 1.0400 - acc: 0.6525     
Epoch 4/20
469/468 [==============================] - 9s - loss: 0.8114 - acc: 0.7314     
Epoch 5/20
469/468 [==============================] - 8s - loss: 0.6775 - acc: 0.7755     
Epoch 6/20
469/468 [==============================] - 9s - loss: 0.5811 - acc: 0.8106     
Epoch 7/20
469/468 [==============================] - 9s - loss: 0.5152 - acc: 0.8319     
Epoch 8/20
469/468 [==============================] - 8s - loss: 0.4729 - acc: 0.8483     
Epoch 9/20
469/468 [==============================] - 9s - loss: 0.4182 - acc: 0.8635     
Epoch 10/

In [45]:
loss = 0
for i in range(10):
    loss = loss+score[i][0]
loss=loss/10
accu = 0
for i in range(10):
    accu = accu+score[i][1]
accu=accu/10
print('Test loss:', loss)
print('Test accuracy:', accu)

Test loss: 0.124701137907
Test accuracy: 0.96165
